In [1]:
import pandas as pd
import os
from itertools import product
from tqdm import tqdm 
from joblib import Parallel, delayed

## Load and group data

In [2]:
folder = "C:/Users/Oscar/Documents/Proyecto_Data_Science/"
file = "sales_train.csv"
out_file = 'df2.csv'

if os.path.exists(os.path.join(folder, out_file)):
    
    df2 = pd.read_csv(os.path.join(folder, out_file))
    
else:                

    df = pd.read_csv(os.path.join(folder, file))
    df['date'] = pd.to_datetime(df.date)
    df['month'] = df.date.dt.to_period('M').astype(str)

    df2 = df.groupby(['shop_id', 'item_id', 'month']).item_cnt_day.sum()
    df2.columns = ['shop_id', 'item_id', 'month', 'item_cnt_month']
    df2 = df2.reset_index()
    df2.to_csv(os.path.join(folder, out_file), index=False)    

In [13]:
shops = df2.shop_id.unique()
items = df2.item_id.unique()
new_index = list(map(lambda x: '2013-%02d' % x, range(1, 13)))

iterables = list(product(shops, items))

In [4]:
def add_missing_months(df, shop, item):
    subdf = df[(df.shop_id == shop) & (df.item_id == item)]
    subdf = subdf.set_index('month').reindex(new_index, fill_value = 0)
    subdf['shop_id'] = shop
    subdf['item_id'] = item
    return subdf

def pool_add_missing_months(df, chunk):
    for shop, item in tqdm(chunk):
        subdf = add_missing_months(df2, shop, item).reset_index()
        subdf.to_csv(os.path.join(folder, out_file_final), header=False, index=False, mode='a')

In [5]:
out_file_final = 'sales_train_month.csv'

df3 = pd.DataFrame(columns=['month', 'shop_id', 'item_id', 'item_cnt_month'])
df3.to_csv(os.path.join(folder, out_file_final), index=False)

In [17]:
n_chunks = 1000
chunk_len = int(len(iterables) / n_chunks)
start = 0
end = chunk_len

chunks = []

for i in range(n_chunks):
    chunks.append(iterables[start:end:chunk_len])
    start += chunk_len
    end + chunk_len

In [18]:
len(chunks)

1000

In [22]:
iterables[start:end]

[]